# Fraud Detection - Gold Layer

## Purpose
Create analytics-ready, denormalized tables by joining silver layer tables for:
- Machine learning model training
- Business intelligence and reporting
- Feature-rich datasets with all relevant dimensions

## Gold Tables Created
1. `tx_train_gold` - Training dataset (labeled transactions + all dimensions)
2. `tx_score_gold` - Scoring dataset (unlabeled transactions + all dimensions)

## Data Flow
Silver Tables → Join Dimensions → Gold Tables (denormalized, analytics-ready)

## Joins Performed
- Transactions + Labels (inner join for train, left anti for score)
- Transactions + Users (left join on client_id)
- Transactions + Cards (left join on card_id)
- Transactions + MCC (left join on mcc code)

In [0]:
from pyspark.sql.functions import broadcast

# Load silver tables
tx_train = spark.table("workspace.fraud.tx_train_silver")  # Already has labels joined
usr = spark.table("workspace.fraud.users_silver")
crd = spark.table("workspace.fraud.cards_silver")
mcc = spark.table("workspace.fraud.mcc_dim_silver")

# Create denormalized gold table with all dimensions
tx_train_gold = (
    tx_train
      .join(broadcast(usr), tx_train.client_id == usr.id, "left")
      .drop(usr.id)  # Drop user id to avoid duplicate
      .join(broadcast(crd), tx_train.card_id == crd.id, "left")
      .drop(crd.id, crd.client_id)  # Drop card id and client_id to avoid duplicates
      .join(broadcast(mcc), "mcc", "left")  # Join on mcc code
)

# Save to gold table
tx_train_gold.write.mode("overwrite").option("overwriteSchema", "true").format("delta") \
    .saveAsTable("workspace.fraud.tx_train_gold")

print(f"Created tx_train_gold with {tx_train_gold.count():,} rows")

In [0]:
from pyspark.sql.functions import broadcast

# Load silver tables
tx_score = spark.table("workspace.fraud.tx_score_silver")  # Unlabeled transactions
usr = spark.table("workspace.fraud.users_silver")
crd = spark.table("workspace.fraud.cards_silver")
mcc = spark.table("workspace.fraud.mcc_dim_silver")

# Create denormalized gold table with all dimensions
tx_score_gold = (
    tx_score
      .join(broadcast(usr), tx_score.client_id == usr.id, "left")
      .drop(usr.id)  # Drop user id to avoid duplicate
      .join(broadcast(crd), tx_score.card_id == crd.id, "left")
      .drop(crd.id, crd.client_id)  # Drop card id and client_id to avoid duplicates
      .join(broadcast(mcc), "mcc", "left")  # Join on mcc code
)

# Save to gold table
tx_score_gold.write.mode("overwrite").option("overwriteSchema", "true").format("delta") \
    .saveAsTable("workspace.fraud.tx_score_gold")

print(f"Created tx_score_gold with {tx_score_gold.count():,} rows")

---
## Verification
Verify gold tables were created successfully.

In [0]:
# Summary of all gold tables
print("Gold Layer Tables Summary")
print("=" * 60)

gold_tables = [
    "tx_train_gold",
    "tx_score_gold"
]

for table in gold_tables:
    try:
        count = spark.sql(f"SELECT COUNT(*) as cnt FROM workspace.fraud.{table}").collect()[0]['cnt']
        print(f"{table:25} {count:>15,} rows")
    except Exception as e:
        print(f"{table:25} ERROR: {str(e)[:30]}")

print("=" * 60)
print()

# Show label distribution in training set
print("Label Distribution in tx_train_gold:")
spark.sql("SELECT label, COUNT(*) as count FROM workspace.fraud.tx_train_gold GROUP BY label ORDER BY label").show()

# Show column count
print(f"tx_train_gold has {len(spark.table('workspace.fraud.tx_train_gold').columns)} columns (denormalized)")
print(f"tx_score_gold has {len(spark.table('workspace.fraud.tx_score_gold').columns)} columns (denormalized)")

---
## Geographic Features
Verify that geographic categorization features from Silver layer are included in Gold tables:
- `location_type`: Categorizes transactions as US_State, International, Online, or Unknown
- `is_cross_border`: Boolean flag for international transactions

---
## Geographic Features
Verify that geographic categorization features from Silver layer are included in Gold tables:
- `location_type`: Categorizes transactions as US_State, International, Online, or Unknown
- `is_cross_border`: Boolean flag for international transactions

In [0]:
from pyspark.sql import functions as F

print("Geographic Features in Gold Tables")
print("=" * 80)

# Check if geographic columns exist
train_cols = spark.table('workspace.fraud.tx_train_gold').columns
print(f"\n✓ location_type in tx_train_gold: {'location_type' in train_cols}")
print(f"✓ is_cross_border in tx_train_gold: {'is_cross_border' in train_cols}")

# Show distribution in training set
print("\n1. Location Type Distribution (Training Set):")
spark.table('workspace.fraud.tx_train_gold') \
    .groupBy('location_type').count() \
    .orderBy(F.desc('count')).show()

print("\n2. Cross-Border Transaction Summary (Training Set):")
spark.table('workspace.fraud.tx_train_gold') \
    .groupBy('is_cross_border').count().show()

# Fraud rate by location type
print("\n3. Fraud Rate by Location Type:")
spark.table('workspace.fraud.tx_train_gold') \
    .groupBy('location_type') \
    .agg(
        F.count('*').alias('total_txns'),
        F.sum(F.when(F.col('label') == 'Yes', 1).otherwise(0)).alias('fraud_count')
    ) \
    .withColumn('fraud_rate_pct', F.round((F.col('fraud_count') / F.col('total_txns') * 100), 2)) \
    .orderBy(F.desc('fraud_rate_pct')).show(truncate=False)

print("\n4. Top International Locations (Fraud Cases):")
spark.table('workspace.fraud.tx_train_gold') \
    .filter((F.col('location_type') == 'International') & (F.col('label') == 'Yes')) \
    .groupBy('merchant_state', 'merchant_city') \
    .count() \
    .orderBy(F.desc('count')).show(10, truncate=False)

print("\n✅ Geographic features successfully included in Gold layer!")

In [0]:
# Show sample records with geographic features
print("Sample Records with Geographic Features:")
print("=" * 80)

spark.table('workspace.fraud.tx_train_gold') \
    .select(
        'tx_id', 'merchant_state', 'merchant_city', 
        'location_type', 'is_cross_border', 'amount', 'label'
    ) \
    .filter(F.col('location_type') == 'International') \
    .show(10, truncate=False)